<a href="https://colab.research.google.com/github/julianovale/project_trains/blob/master/Exemplo_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import I, Matrix, symbols, Symbol, eye
from datetime import datetime
import numpy as np
import pandas as pd


In [ ]:
# Rotas
R1 = Matrix([[0,"R1_p1",0],[0,0,"R1_v1"],[0,0,0]])
R2 = Matrix([[0,"R2_p1",0],[0,0,"R2_v1"],[0,0,0]])

In [ ]:
# Seções (semáforos)
T1 = Matrix([[0, "p1"],["v1", 0]])

In [ ]:
def kronSum(A,B):
  m = np.size(A,1)
  n = np.size(B,1)
  A = np.kron(A,np.eye(n))
  B = np.kron(np.eye(m),B)
  return A + B

In [ ]:
momento_inicio = datetime.now() 
'''
Algebra de rotas
'''
rotas = kronSum(R1,R2)

'''
Algebra de seções 
secoes = kronSum(T1,T2)
secoes = kronSum(secoes,T3)
secoes = kronSum(secoes,T4)
secoes = kronSum(secoes,T5)
'''

'''
Algebra de sistema
'''
sistema = np.kron(rotas, T1) # lembrar de trocar para "secoes" se tiver vários semáforos

# calcula tempo de processamento
tempo_processamento = datetime.now() - momento_inicio

In [ ]:
sistema = pd.DataFrame(data=sistema,index=list(range(1,np.size(sistema,0)+1)), columns=list(range(1,np.size(sistema,1)+1)))

In [ ]:
sistema.shape

(18, 18)

In [ ]:
print(tempo_processamento)

0:00:00.016545


In [ ]:
sistema

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0,0,0,1.0*R2_p1*p1,0,0,0,1.0*R1_p1*p1,0,0,0,0,0,0,0,0,0,0
2,0,0,1.0*R2_p1*v1,0,0,0,1.0*R1_p1*v1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1.0*R2_v1*p1,0,0,0,1.0*R1_p1*p1,0,0,0,0,0,0,0,0
4,0,0,0,0,1.0*R2_v1*v1,0,0,0,1.0*R1_p1*v1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,1.0*R1_p1*p1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,1.0*R1_p1*v1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1.0*R2_p1*p1,0,0,0,1.0*R1_v1*p1,0,0,0,0
8,0,0,0,0,0,0,0,0,1.0*R2_p1*v1,0,0,0,1.0*R1_v1*v1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1.0*R2_v1*p1,0,0,0,1.0*R1_v1*p1,0,0
10,0,0,0,0,0,0,0,0,0,0,1.0*R2_v1*v1,0,0,0,1.0*R1_v1*v1,0,0,0


In [ ]:
momento_inicio = datetime.now() 
colunas = ['de_noh', 'para_noh', 'aresta']
grafo = pd.DataFrame(columns=colunas)
r = 1
c = 1
for j in range(np.size(sistema,0)):
  for i in range(np.size(sistema,0)):
    if sistema.loc[r,c]==0 and c < np.size(sistema,0):
      c += 1
    elif c < np.size(sistema,0):
      grafo.loc[len(grafo)+1] = (r, c, sistema.loc[r,c])  
      c += 1
    else:
      c = 1
      r += 1
tempo_processamento = datetime.now() - momento_inicio
print(tempo_processamento)

0:00:00.090842


In [ ]:
grafo['aresta'] = grafo['aresta'].astype('str')
grafo

,de_noh,para_noh,aresta
1,1,4,1.0*R2_p1*p1
2,1,8,1.0*R1_p1*p1
3,2,3,1.0*R2_p1*v1
4,2,7,1.0*R1_p1*v1
5,3,6,1.0*R2_v1*p1
6,3,10,1.0*R1_p1*p1
7,4,5,1.0*R2_v1*v1
8,4,9,1.0*R1_p1*v1
9,5,12,1.0*R1_p1*p1
10,6,11,1.0*R1_p1*v1


In [ ]:
new = grafo["aresta"].str.split("*", n = -1, expand = True) 
grafo["aresta"]=new[1]
grafo["semaforo_secao"]=new[2]
new = grafo["aresta"].str.split("_", n = -1, expand = True) 
grafo["semaforo_trem"]=new[1]
grafo['coincide'] = np.where(grafo['semaforo_secao']==grafo['semaforo_trem'], True, False)
grafo


,de_noh,para_noh,aresta,semaforo_secao,semaforo_trem,coincide
1,1,4,R2_p1,p1,p1,True
2,1,8,R1_p1,p1,p1,True
3,2,3,R2_p1,v1,p1,False
4,2,7,R1_p1,v1,p1,False
5,3,6,R2_v1,p1,v1,False
6,3,10,R1_p1,p1,p1,True
7,4,5,R2_v1,v1,v1,True
8,4,9,R1_p1,v1,p1,False
9,5,12,R1_p1,p1,p1,True
10,6,11,R1_p1,v1,p1,False


In [ ]:
grafo.drop(grafo[grafo.coincide == False].index, inplace=True)
grafo

,de_noh,para_noh,aresta,semaforo_secao,semaforo_trem,coincide
1,1,4,R2_p1,p1,p1,True
2,1,8,R1_p1,p1,p1,True
6,3,10,R1_p1,p1,p1,True
7,4,5,R2_v1,v1,v1,True
9,5,12,R1_p1,p1,p1,True
11,7,10,R2_p1,p1,p1,True
14,8,13,R1_v1,v1,v1,True
17,10,11,R2_v1,v1,v1,True
18,10,15,R1_v1,v1,v1,True
19,12,17,R1_v1,v1,v1,True
